In [ ]:
from lec_utils import *
import lec17_util as util


<div class="alert alert-info" markdown="1">

#### Lecture 17

# Pipelines

### EECS 398: Practical Data Science, Spring 2025

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/sp25">github.com/practicaldsc/sp25</a> • 📣 See latest announcements [**here on Ed**](https://edstem.org/us/courses/78535/discussion/6647877) </small>
    
</div>


### Agenda 📆

- `OneHotEncoder` and multicollinearity.
- Pipelines🚰.
- Generalization 🔭.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

## `OneHotEncoder` and multicollinearity

---

### Example: Commute times 🚗

- Let's reload our trusty commute times dataset.

In [ ]:
df = pd.read_csv('data/commute-times.csv')
df['day_of_month'] = pd.to_datetime(df['date']).dt.day
df['month'] = pd.to_datetime(df['date']).dt.month_name()
df.head()

- We'll focus specifically on the `'day'` and `'month'` columns.

In [ ]:
df[['day', 'month']]

### Example transformer: `OneHotEncoder`

- Last class, we had to manually one hot encode the `'day'` column. Let's figure out how to one hot encode it automatically, along with the new `'month'` column.

In [ ]:
df[['day', 'month']]

- First, we need to import the relevant class from `sklearn.preprocessing`.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

- Like with `StandardScaler`, we need to instantiate **and fit** our `OneHotEncoder` instsance before it can transform anything.

In [ ]:
ohe = OneHotEncoder()

In [ ]:
ohe.fit(df[['day', 'month']])

- Once we've fit, when we use the `transform` method, we get a result we might not expect.

In [ ]:
ohe.transform(df[['day', 'month']])

- Since the resulting matrix is **sparse** – most of its elements are 0 – `sklearn` uses a more efficient representation than a regular `numpy` array. We can convert to a regular (dense) array:

In [ ]:
ohe.transform(df[['day', 'month']]).toarray()

- The column names from `df[['day', 'month']]` don't appear in the output above. We can use the `get_feature_names_out` method on `ohe` to access the names and order of the one hot encoded columns, though:

In [ ]:
ohe.get_feature_names_out()

In [ ]:
pd.DataFrame(ohe.transform(df[['day', 'month']]).toarray(), 
             columns=ohe.get_feature_names_out()) # If we need a DataFrame back, for some reason.

- Usually, we won't perform all of these intermediate steps, since the `OneHotEncoder` will be part of a larger **Pipeline**.

### Example: Heights and weights

- We now know how to use `OneHotEncoder`.

- To illustrate a mathematical issue involving one hot encoding, let's load in another dataset, this time containing the weights and heights of 25,000 18 year olds, taken from [here](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_Dinov_020108_HeightsWeights).

In [ ]:
people = pd.read_csv('data/heights-weights.csv').drop(columns=['Index'])
people.head()

In [ ]:
people.plot(kind='scatter', x='Height (Inches)', y='Weight (Pounds)', 
            title='Weight vs. Height for 25,000 18 Year Olds')

### Motivating example

- Suppose we fit a simple linear regression model that uses **height in inches**, $x$ to predict **weight in pounds**, $y$.

$$\text{predicted weight}_i = w_0 + w_1 \cdot \text{height in inches}_i$$

In [ ]:
X = people[['Height (Inches)']]
y = people['Weight (Pounds)']

In [ ]:
from sklearn.linear_model import LinearRegression
people_one_feat = LinearRegression()
people_one_feat.fit(X, y)

- $w_0^*$ and $w_1^*$ are shown below, along with the model's MSE on the data we used to train it.<br><small>We call this the model's **training MSE**.</small>

In [ ]:
people_one_feat.intercept_, people_one_feat.coef_

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y, people_one_feat.predict(X))

### An added feature

- Now, suppose we fit another regression model, that uses **height in inches** AND **height in feet** to predict weight.

$$\text{predicted weight}_i = w_0 + w_1 \cdot \text{height in inches}_i + w_2 \cdot \text{height in feet}_i$$

In [ ]:
people['Height (Feet)'] = people['Height (Inches)'] / 12 # 12 inches = 1 foot.

In [ ]:
X2 = people[['Height (Inches)', 'Height (Feet)']]
X2

In [ ]:
people_two_feat = LinearRegression()
people_two_feat.fit(X2, y)

- What are $w_0^*$, $w_1^*$, $w_2^*$, and the model's MSE?

In [ ]:
people_two_feat.intercept_, people_two_feat.coef_

In [ ]:
mean_squared_error(y, people_two_feat.predict(X2))

- **Observation**: The intercept is the same as before (roughly -82.59), as is the MSE. However, the coefficients on `'Height (Inches)'` and `'Height (Feet)'` are massive in size!

- It should be unsurprising that the MSE is the same, because the span of the design matrix is the same. So, the best predictions should be the same, too.

- But what's going on with the coefficients?

### Redundant features

- Suppose in the first model, $w_0^* = -80$ and $w_1^* = 3$.

$$\text{predicted weight}_i = -80 + 3 \cdot \text{height in inches}_i$$

- In the second model, we have:

$$\begin{align*}\text{predicted weight}_i &= w_0^* + w_1^* \cdot \text{height in inches}_i + w_2^* \cdot \text{height in feet}_i \end{align*}$$

- But, since $\text{height in feet}_i = \frac{\text{height in inches}_i}{12}$:

$$\begin{align*}\text{predicted weight}_i &= w_0^* + w_1^* \cdot \text{height in inches}_i + w_2^* \cdot \text{height in feet}_i \\ &= w_0^* + w_1^* \cdot \text{height in inches}_i + w_2^* \cdot \left( \frac{\text{height in inches}_i}{12} \right) \\ &= w_0^* + \left( w_1^* + \frac{w_2^*}{12} \right) \cdot \text{height in inches}_i \end{align*}$$

- In the first model, we already found the "best" intercept ($-80$) and slope ($3$) in a linear model that uses height in inches to predict weight.

- **So, as long as $w_1^* + \frac{w_2^*}{12} = 3$ in the second model, the second model's predictions will be the same as the first, and hence they will also minimize MSE.**

### Infinitely many parameter choices

- **Issue**: There are an infinite number of $w_1^*$ and $w_2^*$ that satisfy $w_1^* + \frac{w_2^*}{12} = 3$!

$$\begin{align*}\text{predicted weight}_i &= -80 + 5 \cdot \text{height in inches}_i - 24 \cdot \text{height in feet}_i \end{align*}$$

$$\begin{align*}\text{predicted weight}_i &= -80 - 1 \cdot \text{height in inches}_i + 48 \cdot \text{height in feet}_i \end{align*}$$

- Both hypothesis functions look very different, but actually make the same predictions.

- `model.coef_` could return either set of coefficients, or any other of the infinitely many options. 

- But neither set of coefficients is **has any meaning!**

In [ ]:
-80 + 5 * people['Height (Inches)'] - 24 * people['Height (Feet)']

In [ ]:
-80 - 1 * people['Height (Inches)'] + 48 * people['Height (Feet)']

### Multicollinearity

- Multicollinearity occurs when features in a regression model are **highly correlated** with one another.<br><small>In other words, multicollinearity occurs when **a feature can be predicted using a linear combination of other features, fairly accurately**.</small>

- When multicollinearity is present in the features, the **coefficients in the model** are uninterpretable – they have no meaning.<br><small>A "slope" represents "the rate of change of $y$ with respect to a feature", when all other features are held constant – but if there's multicollinearity, you can't hold other features constant.</small>

- **Note: Multicollinearity doesn't impact a model's predictions!**
    - It doesn't impact a model's ability to generalize to unseen data.
    - If features are multicollinear in the data we've seen, they will probably be multicollinear in the data we haven't seen, drawn from the same distribution.

- **Solutions**:
    - Manually remove highly correlated features.
    - Use a dimensionality reduction technique (such as PCA) to automatically reduce dimensions.

### One hot encoding and multicollinearity

- **One hot encoding will result in multicollinearity unless you drop one of the one hot encoded features.**

- Suppose we have the following fitted model:<br><small>For illustration, assume `'weekend'` was originally a categorical feature with two possible values, `'Yes'` or `'No'`.

$$
\begin{aligned}
H(\vec x_i) = 1 - 3 \cdot \text{departure hour}_i + 2 \cdot (\text{weekend}_i==\text{Yes}) - 2 \cdot (\text{weekend}_i==\text{No})
\end{aligned}
$$

- This is equivalent to:

$$
\begin{aligned}
H(\vec x_i) = 10 - 3 \cdot \text{departure hour}_i - 7 \cdot (\text{weekend}_i==\text{Yes}) - 11 \cdot (\text{weekend}_i==\text{No})
\end{aligned}
$$

- Note that for a particular row in the dataset, $\text{weekend}_i==\text{Yes} + \text{weekend}_i==\text{No}$ is always equal to 1.

$$X = \begin{bmatrix} 1 & 8.45 & 0 & 1 \\
                  1 & 11 & 0 & 1 \\
                  1 & 7.39 & 1 & 0 \\
                  1 & 9.98 & 1 & 0 \\
                  1 & 10.45 & 0 & 1 \\\end{bmatrix}$$
                  
<center><small>A possible design matrix for this model.</small></center>

- What's the issue with the example design matrix above?<br><small>See the annotated slides.</small>

### One hot encoding and multicollinearity

$$X = \begin{bmatrix} 1 & 8.45 & 0 & 1 \\
                  1 & 11 & 0 & 1 \\
                  1 & 7.39 & 1 & 0 \\
                  1 & 9.98 & 1 & 0 \\
                  1 & 10.45 & 0 & 1 \\\end{bmatrix}$$
                  
<center><small>A possible design matrix for this model.</small></center>

- The columns of the design matrix, $X$ above are **not** linearly independent!
The column of all 1s can be written as a linear combination of the $\text{weekend==Yes}$ and $\text{weekend==No}$ columns.

$$\text{column 1} = \text{column 3} + \text{column 4}$$

- This means that the design matrix is not **full rank**, which means that $X^TX$ is **not invertible**.

- This means that there are **infinitely many possible solutions $\vec{w}^*$ to the normal equations, $(X^TX) \vec{w} = X^T\vec{y}$**!<br><small>That's a problem, because we don't know which of these infinitely many solutions `model.coef_` will find for us, and it's impossible to interpret the resulting coefficients, as we saw two slides ago.</small>

- **Solution**: Drop one of the one hot encoded columns. `OneHotEncoder` has an option to do this.

### `OneHotEncoder` returns

- Let's switch back to the commute times dataset, `df`.

In [ ]:
df[['day', 'month']]

- Let's try using `drop='first'` when instantiating a `OneHotEncoder`.

In [ ]:
ohe_drop_one = OneHotEncoder(drop='first')

In [ ]:
ohe_drop_one.fit(df[['day', 'month']])

- How many features did the resulting transformer create?

In [ ]:
len(ohe_drop_one.get_feature_names_out())

- Where did this number come from?

In [ ]:
df['day'].nunique()

In [ ]:
df['month'].nunique()

### Key takeaways

- Multicollinearity is present in a linear model when one feature can be accurately predicted using one or more other features.<br><small>In other words, it is present when a feature is **redundant**.</small>

- Multicollinearity doesn't pose an issue for prediction; it doesn't hinder a model's ability to generalize. Instead, it renders the **coefficients** of a linear model meaningless.

## Pipelines🚰

---

### Recap: Commute times 🚗

In [ ]:
(
    df
    .plot(kind='scatter', x='departure_hour', y='minutes')
    .update_layout(xaxis_title='Home Departure Time (AM)', 
                   yaxis_title='Minutes',
                   title='Commuting Time vs. Home Departure Time')
)

- So far, our goal has been to predict commute time in `'minutes'`, given `'departure_hour'`.

- We just learned how to use `OneHotEncoder` to encode `'day'` and `'month'` as numerical columns.<br> We'll look at how we can **easily** use these columns – and more! – as inputs to a linear model that predicts commute times.

### Pipelines in `sklearn`

- From [`sklearn`'s documentation](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html):

> `Pipeline` allows you to sequentially apply a list of transformers to preprocess the data and, **if desired**, conclude the sequence with a final predictor for predictive modeling.<br><br>Intermediate steps of the pipeline must be "transforms", that is, they must implement `fit` and `transform` methods. The final estimator only needs to implement `fit`.

- General template: `pl = make_pipeline(transformer_1, transformer_2, ..., model)`.<br><small>Note that the `model` is optional, meaning you can have Pipelines of just transformers.

- Once a Pipeline is instantiated, you can fit **all** steps (transformers and model) using `pl.fit(X, y)`.

- To make predictions using **raw, untransformed data**, use `pl.predict(X)`.

### Our first Pipeline

- Let's build a Pipeline that:
    1. One hot encodes `'day'` and `'month'`.
    2. Fits a regression model on just the one hot encoded data.

In [ ]:
# You can either use the Pipeline class constructor directly,
# or the make_pipeline helper function (my preference).
from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
...

- Now that `pl` is instantiated, we `fit` it the same way we would fit the individual steps.

In [ ]:
...

- Now, to make predictions using **raw data**, all we need to do is use `pl.predict`:

In [ ]:
...

- `pl` performs **both** feature transformation and prediction with just a single call to `predict`!

<div class="alert alert-danger">
    
#### Reference Slide
    
### Pipeline internals
    
</div>

- We can access individual "steps" of a `Pipeline` through the `named_steps` attribute.

In [ ]:
# These names are automatically generated by make_pipeline.
# If you use the Pipeline() constructor,
# you can choose these names yourself.
pl.named_steps

In [ ]:
pl.named_steps['onehotencoder'].transform(df[['day', 'month']]).toarray()

In [ ]:
pl.named_steps['onehotencoder'].get_feature_names_out()

In [ ]:
pl.named_steps['linearregression'].coef_

### More sophisticated Pipelines

- In the previous slide, we one hot encoded every input column, and didn't use any columns that were originally numeric, i.e. we didn't use `'departure_hour'`.<br><small>That's not realistic or useful!</small>

- What if we want to perform different transformations on different columns, or include some columns without transformation?

- Or, what if we want to perform multiple transformations to the same column?

- There are a variety of useful functions/classes we can use:

| Name | Functionality |
| --- | --- |
| `ColumnTransformer` | Allows us to transform different columns with different transformations.<br><small>Instantiate a `ColumnTransformer` using a list of tuples, where:<br>• The first element is a "name" we choose for the transformer.<br>• The second element is a transformer instance (e.g. `OneHotEncoder()`).<br>• The third element is a **list of relevant column names**.</small> | 
| `FunctionTransformer` | Allows us to create a custom transformation (similar to using `.apply` on a DataFrame's columns). |
| `make_pipeline` | Helper function for creating a `Pipeline` (slightly less verbose).<br>**Note that you can make a pipeline of just transformations,<br> if you want to use multiple transformations on the same column!** |
| `make_column_transformer` | Helper function for creating a `ColumnTransformer`. |

### The plan

- Before writing any code, let's plan out _how_ we want to transform our data.

In [ ]:
df[['departure_hour', 'day', 'month', 'day_of_month']]

- `'departure_hour'`: Create degree 2 and degree 3 **polynomial features**:

$$H(\vec x_i) = ... + w_1 \cdot \text{departure hour}_i + w_2 \cdot \left(\text{departure hour}_i\right)^2 + w_3 \cdot \left( \text{departure hour}_i \right)^3 + ...$$

- `'day'`: One hot encode.

- `'month'`: One hot encode.

- `'day_of_month'`: Separate into five weeks, then one hot encode.<br>
<small>Days 1 to 7 are Week 1, Days 8 to 15 are Week 2, and so on.</small>

- After all of these transformations, we'll fit a `LinearRegression` object – i.e., fit a linear model.

<center><small>
    
`'departure_hour'`: Create degree 2 and degree 3 polynomial features.<br>
`'day'`: One hot encode.<br>
`'month'`: One hot encode.<br>
`'day_of_month'`: Separate into five weeks, then one hot encode.<br>
    
</small></center>

- Let's start with `'day_of_month'`, since it seems to involve the most complicated transformations.

- First, let's figure out how to extract the week number given the day of the month.

In [ ]:
example_vals = df['day_of_month'].tail()
example_vals

In [ ]:
# Expression to convert from day of month to Week #.
...

In [ ]:
# The function that FunctionTransformer takes in
# itself takes in a Series/DataFrame, not a single element!
# Here, we're having that function return a new Series/DataFrame,
# depending on what's passed in to .tranform (experiment on your own).
from sklearn.pipeline import FunctionTransformer
week_converter = ...

In [ ]:
week_converter.transform(df[['day_of_month']])

- We need to apply two consecutive transformations to `'day_of_month'`, which calls for a Pipeline.

In [ ]:
day_of_month_transformer = ...
day_of_month_transformer

In [ ]:
day_of_month_transformer.fit_transform(df[['day_of_month']]).toarray()

- So, `day_of_month_transformer` does everything we need to transform `'day_of_month'`.

<center><small>
    
`'departure_hour'`: Create degree 2 and degree 3 polynomial features.<br>
`'day'`: One hot encode.<br>
`'month'`: One hot encode.<br>
`'day_of_month'`: Separate into five weeks, then one hot encode. ✅ **Use `day_of_month_transformer`.**<br>
    
</small></center>

- Every other column only needs a single transformation.<br>**To specify which transformations to apply to which columns, create a `ColumnTransformer`**.

In [ ]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
preprocessing = make_column_transformer(
    (PolynomialFeatures(3, include_bias=False), ['departure_hour']),
    (OneHotEncoder(drop='first'), ['day', 'month']),
    (day_of_month_transformer, ['day_of_month']),
    remainder='drop'
)
preprocessing

- Now, we're ready for a final Pipeline!

In [ ]:
model = make_pipeline(preprocessing, LinearRegression())
model

In [ ]:
model.fit(X=df[['departure_hour', 'day', 'month', 'day_of_month']], y=df['minutes'])

### The punchline

- Now that our Pipeline is fit, we can use it to make predictions using **raw data**!<br><small>What's the predicted commute time if I leave at 8:30AM on a Wednesday in March, which happens to be the 19th of the month?</small>

In [ ]:
...

- Note that when calling `model.predict`, I didn't need to think about one hot encoding, or polynomial features, or any other aspects of the feature engineering process.

<div class="alert alert-success">
<h3>Activity</h3>
    
How many columns does the final design matrix that `model` creates have? If you write code to determine the answer, make sure you can walk through the steps over the past few slides to figure out **why** the answer is what it is.

In [ ]:
model

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
What questions do you have?

## Generalization 🔭

---

### Motivation

- You and Billy are studying for an upcoming exam. You both decide to test your understanding by taking a **practice exam**.<br><small>Your logic: If you do well on the practice exam, you should do well on the real exam.</small>

- You each take the practice exam once and look at the solutions afterwards.

- **Your strategy**: Memorize the answers to all practice exam questions, e.g. "Question 1: A; Question 2: C; Question 3: A."

- **Billy's strategy**: Learn high-level concepts from the solutions, e.g. "the TF-IDF of term $t$ in document $d$ is large when $t$ occurs often in $d$ but rarely overall."

- Who will do better on the **practice exam**? Who will probably do better on the **real exam**? 🧐

### Evaluating the quality of a model

- So far, we've computed the MSE of our fit regression models on the **data that we used to fit them**, i.e. the **training data**.<br><small>This mean squared error is called the **training MSE**, or **training error**.</small>

- We've said that Model A is **better** than Model B if Model A's MSE is **lower** than Model B's MSE.
    - Remember, our **training data** is a sample from some population.
    - Just because a model fits the training data well doesn't mean it will **generalize** and work well on **similar, unseen samples** from the same population!

### Overfitting and underfitting

- Let's collect two samples $\{(x_i, y_i)\}$ from the same population.

In [ ]:
np.random.seed(23) # For reproducibility.
def sample_from_pop(n=100):
    x = np.linspace(-2, 3, n)
    y = x ** 3 + (np.random.normal(0, 3, size=n))
    return pd.DataFrame({'x': x, 'y': y})
sample_1 = sample_from_pop()
sample_2 = sample_from_pop()

- For now, let's just look at Sample 1. The relationship between $x$ and $y$ is roughly **cubic**; that is, $y \approx x^3$.<br><small>Remember, in reality, you won't get to see the population distribution. If you could, there'd be no need to build a model!</small>

In [ ]:
px.scatter(sample_1, x='x', y='y', title='Sample 1')

### Polynomial regression

- Let's fit three **polynomial** models on Sample 1: degree 1, degree 3, and degree 25.<br><small>Again, we'll use the `PolynomialFeatures` transformer.</small>

In [ ]:
# fit_transform fits and transforms the same input.
# We tell it not to add a column of 1s, because
# LinearRegression() does this automatically later on.
d2 = PolynomialFeatures(3, include_bias=False)
d2.fit_transform(np.array([1, 2, 3, 4, -2]).reshape(-1, 1))

- Below, we look at our three models' predictions on Sample 1, which they were **trained** on.

In [ ]:
# Look at the definition of train_and_plot in lec17_util.py if you're curious as to how the plotting works.
fig = util.train_and_plot(train_sample=sample_1, test_sample=sample_1, degs=[1, 3, 25], data_name='Sample 1')
fig.update_layout(title='Trained on Sample 1, Performance on Sample 1')

- The degree 25 polynomial has the lowest MSE on Sample 1.

- How do the same fit polynomials look on Sample 2?

In [ ]:
fig = util.train_and_plot(train_sample=sample_1, test_sample=sample_2, degs=[1, 3, 25], data_name='Sample 2')
fig.update_layout(title='Trained on Sample 1, Performance on Sample 2')

- The degree 3 polynomial has the lowest MSE on Sample 2. 

- Note that **we didn't get to see Sample 2 when fitting our models**! 

- As such, it seems that the degree 3 polynomial **generalizes better** to unseen data than the degree 25 polynomial does.

- What if we fit a degree 1, degree 3, and degree 25 polynomial **on Sample 2** as well?

In [ ]:
util.plot_multiple_models(sample_1, sample_2, degs=[1, 3, 25])

- **Key idea**: Degree 25 polynomials seem to **vary more when trained on different samples** than degree 3 and 1 polynomials do.

- More on this next class!